In [17]:
import pandas as pd
import numpy as np
from sklearn import linear_model
import statsmodels.api as sm
from math import log
from matplotlib import pyplot as plt


In [18]:
#import documents
df=pd.read_csv("world-happiness-report-2021.csv")
df1=pd.read_csv("world-happiness-report.csv")
df1=df1.dropna()
df1

,Country name,year,Life Ladder,Log GDP per capita,Social support,Healthy life expectancy at birth,Freedom to make life choices,Generosity,Perceptions of corruption,Positive affect,Negative affect
0,Afghanistan,2008,3.724,7.370,0.451,50.80,0.718,0.168,0.882,0.518,0.258
1,Afghanistan,2009,4.402,7.540,0.552,51.20,0.679,0.190,0.850,0.584,0.237
2,Afghanistan,2010,4.758,7.647,0.539,51.60,0.600,0.121,0.707,0.618,0.275
3,Afghanistan,2011,3.832,7.620,0.521,51.92,0.496,0.162,0.731,0.611,0.267
4,Afghanistan,2012,3.783,7.705,0.521,52.24,0.531,0.236,0.776,0.710,0.268
...,...,...,...,...,...,...,...,...,...,...,...
1944,Zimbabwe,2016,3.735,7.984,0.768,54.40,0.733,-0.095,0.724,0.738,0.209
1945,Zimbabwe,2017,3.638,8.016,0.754,55.00,0.753,-0.098,0.751,0.806,0.224
1946,Zimbabwe,2018,3.616,8.049,0.775,55.60,0.763,-0.068,0.844,0.710,0.212
1947,Zimbabwe,2019,2.694,7.950,0.759,56.20,0.632,-0.064,0.831,0.716,0.235


In [19]:
newdf= {
    "ladder": np.array(df1["Life Ladder"]),
    "loggedgdp": np.array(df1["Log GDP per capita"]),
    "healthy": np.array(df1["Healthy life expectancy at birth"]),
    "freedom": np.array(df1["Freedom to make life choices"]),
    "corruption": np.array(df1["Perceptions of corruption"]),
    "Social supp": np.array(df1["Social support"]),
    "generosity": np.array(df1["Generosity"])
}

In [20]:
#Regression list
newdf=pd.DataFrame(newdf)
regx=newdf[["healthy","ladder","generosity","corruption"]]
y=newdf["loggedgdp"]




In [21]:
#Regression
x=sm.add_constant(regx)
model=sm.OLS(y,x).fit()
prediction=model.predict(x)

print(model.summary())



                            OLS Regression Results                            
Dep. Variable:              loggedgdp   R-squared:                       0.798
Model:                            OLS   Adj. R-squared:                  0.798
Method:                 Least Squares   F-statistic:                     1684.
Date:                Thu, 01 Dec 2022   Prob (F-statistic):               0.00
Time:                        13:28:07   Log-Likelihood:                -1307.2
No. Observations:                1708   AIC:                             2624.
Df Residuals:                    1703   BIC:                             2652.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.8291      0.141     12.970      0.0

In [22]:

# plt.ylim(0,15)
# plt.xlabel("Perception of corruption")
# plt.ylabel("Logged GDP per capita")
# plt.scatter(newdf["corruption"],y)


In [23]:
#option 1 2SLS regression
n=len(y)
exogenous=newdf[["ladder","generosity","corruption"]]
endogenous=newdf[["healthy"]]
instrument=newdf[["freedom"]]
Z=np.hstack((np.ones((n,1)),exogenous,instrument))
from statsmodels.regression.linear_model import OLS
lm=OLS(endogenous,Z).fit()
lm.summary()
print(n)
print(len(exogenous),len(lm.fittedvalues))

X_hat=np.hstack((np.ones((n,1)),exogenous,lm.fittedvalues.values.reshape(-1,1)))

stage_2=OLS(y,X_hat).fit()
stage_2.summary()

1708
1708 1708


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              loggedgdp   R-squared:                       0.661
Model:                            OLS   Adj. R-squared:                  0.660
Method:                 Least Squares   F-statistic:                     828.3
Date:                Thu, 01 Dec 2022   Prob (F-statistic):               0.00
Time:                        13:28:07   Log-Likelihood:                -1751.5
No. Observations:                1708   AIC:                             3513.
Df Residuals:                    1703   BIC:                             3540.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         19.3346      5.426      3.563      0.000       8.691      29.978
x1             2.8956      0.781      3.707      0.000       1.364       4.428
x2            -3.7697      0.938     -4.021      0.000      -5.609      -1.931
x3            -0.6899      0.211     -3.277      0.001      -1.103      -0.277
x4            -0.3997      0.151     -2.649      0.008      -0.696      -0.104
==============================================================================
Omnibus:                       19.028   Durbin-Watson:                   0.399
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               17.464
Skew:                          -0.203   Prob(JB):                     0.000161
Kurtosis:                       2.716   Cond. No.                     2.17e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.17e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [24]:
#option 2 2SLS regression (used in our paper)
import linearmodels.iv.model as yooo

model_2sls=yooo.IV2SLS(dependent=y,exog=newdf[["ladder","generosity","corruption"]],endog=newdf[["healthy"]],instruments=newdf[["freedom"]]).fit(cov_type = "homoskedastic", debiased = True)
print(model_2sls.summary)


                          IV-2SLS Estimation Summary                          
Dep. Variable:              loggedgdp   R-squared:                      0.9966
Estimator:                    IV-2SLS   Adj. R-squared:                 0.9966
No. Observations:                1708   F-statistic:                 1.238e+05
Date:                Thu, Dec 01 2022   P-value (F-stat)                0.0000
Time:                        13:28:07   Distribution:                F(4,1704)
Cov. Estimator:         homoskedastic                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
ladder         0.2854     0.0481     5.9385     0.0000      0.1912      0.3797
generosity    -0.5235     0.0879    -5.9536     0.00